In [1]:
pip install transformers

In [2]:
pip install huggingface-hub

In [3]:
pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
pip install -i https://test.pypi.org/simple/ bitsandbytes


Looking in indexes: https://test.pypi.org/simple/


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [4]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf', load_in_8bit=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [8]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
prompt = "Hsoub limited company is a "
toks = tokr(prompt, return_tensors="pt")


In [9]:
toks

{'input_ids': tensor([[    1,   379, 29879,   283, 29890,  9078,  5001,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
tokr.batch_decode(toks['input_ids'])

['<s> Hsoub limited company is a ']

In [11]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15)
res


CPU times: user 2.77 s, sys: 480 ms, total: 3.25 s
Wall time: 4.58 s


tensor([[    1,   379, 29879,   283, 29890,  9078,  5001,   338,   263, 29871,
         29896, 29900, 29900, 29995, 15205, 11684,  8819,   653,   310,   379,
         29879,   283, 29890, 21771,   292]], device='cuda:0')

In [12]:
tokr.batch_decode(res)

['<s> Hsoub limited company is a 100% owned subsidiary of Hsoub Holding']

In [48]:
def gen(p, maxlen = 100):
  toks = tokr(p, return_tensors ="pt")
  res = model.generate(**toks.to("cuda"),max_new_tokens=maxlen)
  return tokr.batch_decode(res)


In [24]:
gen(prompt, 50)

['<s> Hsoub limited company is a 100% owned subsidiary of Hsoub group, which is a leading provider of IT services and solutions in the Middle East and North Africa region.\n\nHsoub limited company is a 100% owned']

In [49]:
gen("write me a function in python that can make the robot move forward then backward when he hit something")

["<s> write me a function in python that can make the robot move forward then backward when he hit something\n\nI am using the pyserial library to control a robot and I want to make it move forward and backward based on the sensor readings.\n\nHere is my code so far:\n```\nimport time\nimport serial\n\n# Set the serial port and baud rate\nser = serial.Serial('COM1', 9600)\n\n# Set the robot's forward and backward speeds\nforward_speed = 100\nback"]

In [50]:
response = gen("write me a function in python that can make the robot move forward then backward when he hit something", 700)

In [51]:
decoded_string = response[0].encode('utf-8').decode('unicode-escape')
print(decoded_string)

<s> write me a function in python that can make the robot move forward then backward when he hit something

I am trying to make a simple robot simulation in python and I want the robot to be able to move forward and backward. I have the following code so far:
```
import numpy as np

class Robot:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed = 1

    def move(self, direction):
        if direction == 'forward':
            self.x += self.speed
        elif direction == 'backward':
            self.x -= self.speed

    def hit_wall(self):
        print("Robot hit wall")

# Example usage
robot = Robot(0, 0)
robot.move('forward')
print(robot.x) # Output: 1
robot.move('backward')
print(robot.x) # Output: -1
```
I want to add a function that makes the robot move backward when it hits something, but I am not sure how to do this. Can someone please help me with this?

Here is the code for the robot class:
```
class Robot:
    def __init__(self, x, y):
 

In [44]:
gen(" الروبوتات ",800)

['<s>  الروبوتات والمتابون في المناقشة الوطنية في مصر\n\n====================================================================\n\nThe Role of Robots and Artificial Intelligence in the National Discussion in Egypt\n\n====================================================================\n\nIntroduction\n\nThe rapid advancement of technology has led to the emergence of new concepts and innovations that are changing the way we live, work, and interact with each other. One of the most significant technological advancements in recent years is the development of robots and artificial intelligence (AI). These technologies have the potential to revolutionize various industries, including healthcare, manufacturing, and transportation, among others. However, the integration of robots and AI into society also raises important ethical and societal questions, particularly in the context of the national discussion in Egypt.\n\nIn this article, we will explore the role of robots and AI in the national d

In [45]:
response = gen("ماهي الروبوتات المسلحه",800)

In [46]:
text_with_newlines = response[0].replace("\\n","\n")

print(text_with_newlines)

<s> ماهي الروبوتات المسلحه في العراق

ماهي الروبوتات المسلحه في العراق

The role of armed robots in Iraq

Introduction
------------

Armed robots, also known as unmanned ground vehicles (UGVs), have become increasingly popular in military operations in recent years. These robots are remotely controlled or autonomous and are equipped with a variety of weapons and sensors, making them useful for a range of tasks, including reconnaissance, explosive ordnance disposal (EOD), and combat. In Iraq, armed robots have been used by the military to support operations against ISIS and other terrorist groups. This article will explore the role of armed robots in Iraq and the challenges and benefits of their use.

Challenges and benefits of using armed robots in Iraq
---------------------------------------------------

### Challenges:

1. **Security concerns:** The use of armed robots in Iraq raises significant security concerns, particularly in terms of the potential for hacking or cyber attacks. I